# Ejercicio 2

## Parte 1

In [ ]:
from typing import List

import numpy as np
from IPython.display import display

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import make_scorer, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [ ]:
HOTEL_BOOKINGS_DATASET_PATH = './datasets/hotel_bookings.csv'
hotel_bookings_df = read_data_from_csv(file_path=HOTEL_BOOKINGS_DATASET_PATH)

In [ ]:
hotel_bookings_df

# Información sobre el dataset

## Atributos

| Variable  | Tipo de Variable  |  Descripción  |
|---|---|---|
| hotel  | Variable Cualitativa - Texto - Nominal  | Nombre del hotel  |
|  is_canceled | Variable Cualitativa - Texto - Nominal  |  Valor que indica si la reserva fue cancelada  |
|  lead_time | Variable Cuantitativa - Número - Discreta  | Cantidad de dias entre la reserva y la llegada al hotel  |
|  arrival_date_year | Variable Cuantitativa - Número - Discreta ? |  Año de arrivo |
| arrival_date_month  | Variable Cualitativa - Texto - Nominal ? |  Mes del arrivo |
| arrival_date_week_number  | Variable Cuantitativa - Número - Discreta ?  | Número de la semana del arrivo  |
|  arrival_date_day_of_month | Variable Cuantitativa - Número - Discreta ? | Día del mes del arrivo  |
|  stays_in_weekend_nights  | Variable Cuantitativa - Número - Discreta  |  Días del fin de semana (Sabado - Domingo) que abarca la reserva |
|  stays_in_week_nigths | Variable Cuantitativa - Número - Discreta  | Días de la semana (Lunes a Viernes) que abarca la reserva  |
|  adults | Variable Cuantitativa - Número - Discreta  | Cantidad de adultos  |
|  children | Variable Cuantitativa - Número - Discreta (En el Dataset aparece como continua)  |  Cantidad de niños. Cabe aclarar que los datos provenientes del dataset tiene tipo float y se debe transformar, ya que una reserva no puede tener 1.3 niños  |
| babies  | Variable Cuantitativa - Número - Discreta  | Cantidad de bebes  |
|  meal | Variable Cualitativa - Texto - Nominal  |  Tipo de comida reservada (Detalles en Categorias) |
|  country | Variable Cualitativa - Texto - Nominal  |  Pais de origen  |
| market_segment  | Variable Cualitativa - Texto - Nominal  | Segmento del mercado asignado (Detalles en Categorias)  |
| distribution_channel  |   |   |
|  is_repeated_guest | Variable Cualitativa - Número - Nominal  | Si el huesped ya se hospedo en el hotel  |
|  previous_cancelations |  Variable Cuantitativa - Número - Discreta |  Cantidad de cancelaciones  |
|  previous_bookings_not_canceled |  Variable Cuantitativa - Número - Discreta |  Cantidad de veces que no cancelo la reserva  |
| reserved_room_type  | Variable Cualitativa - Text - Nominal  | Código del tipo de cuarto reservado  |
| booking_changes  |  Variable Cuantitativa - Número - Discreta |  Cantidad de cambios en la reserva desde la fecha de reserva hasta la cancelación o check-in  |
| deposit_types  | Variable Cualitativa - Texto - Discreta  | Tipo de depósito que se hizo (Detalles en Categorias)   |
| agent  | Variable Cualitativa - Número - Nominal  | ID de la agencia que hizo la reserva   |
| company  |  Varaible Cualitativa - Número - Nominal | ID de la compania que hizo la reserva  |
| days_in_waiting_list  |  Variable Cuantitativa - Número - Discreta | Número de días que la reserva estuvo en lista de espera hasta ser confirmada  |
| customer_type  |  Variable Cualitativa - Texto - Nominal |  Tipo de reserva (Detalles en Categorias) |
|  adr |   |   |
|  required_car_parking_spaces |  Variable Cuantitativa - Número - Discreta | Cantidad de espacios de estacionamiento para el huesped  |
| total_of_special_requests  |  Variable Cuantitativa - Número - Discreta | Cantidad de pedidos especiales que hizo el huesped  |
|  reservation_status |  Variable Cualitativa - Texto - Nominal | Estado de la reserva (Detalles en Categorias)  |
|  reservation_status_date | Variable Cuantitativa - Número - Continua  | Fecha de la ultima vez que se actualizo reservation_status  |


## Categorias
- Meal:
    - Undefined/SC : No meal package.
    - BB : Bed & Breakfast.
    - HB : Breakfast and one other meal.
    - FB : Breafast, lunch and dinner.
- Market Segment:
    - TA : Agencias de viaje.
    - TO : Operador turístico.
- Deposit Type:
    - No Deposit : No se hizo un deposito.
    - Non Refund : Un deposito se hizo para el valor total de la reseva.
    - Refundable: Un deposito se hizo para un valor menor al total de la reserva.
- Customer Type:
    - Contract : Cuando la reserva esta asociada a un contrato.
    - Group : Cuando la reserva esta asocidada a un grupo.
    - Transient : Cuando la reserva no esta asociada a un contrato ni a un grupo.
    - Transient-party : Cuando la reserva es del tipo transient pero esta asociada a otra reserva transient.
- Reservation Status:
    - Canceled : La reserva fue cancelada.
    - Check-Out : El cliente hizo el check-in y tambien el check-out.
    - No-Show : El cliente no hizo el check-in pero tampoco cancelo la reserva.

### Analizamos la frecuencia de cada categoria

In [ ]:
def group_by_and_count_category(df: pd.DataFrame, column: str) -> pd.DataFrame:
    group_df = df.groupby([column])[column].count().reset_index(name='count')
    return group_df

def display_count_of_categories(df: pd.DataFrame, list_of_columns: List[str]):
    for categorical_column in list_of_columns:
        display(group_by_and_count_category(df=df, column=categorical_column))

In [ ]:
list_of_category_columns = ['is_canceled', 'meal', 'market_segment', 'reserved_room_type', 'deposit_type', 'customer_type', 'hotel', 'reservation_status', 'assigned_room_type', 'arrival_date_month', 'country', 'distribution_channel']
display_count_of_categories(df=hotel_bookings_df, list_of_columns=list_of_category_columns)

### Analizamos la cantidad de registros vacios

In [ ]:
display(hotel_bookings_df.isnull().sum())

In [ ]:
def calculate_and_display_percentage_of_null_values(df: pd.DataFrame):
    number_of_rows = len(df)
    count_of_nulls: pd.Series = df.isnull().sum()

    for index, count in count_of_nulls.iteritems():
        if count != 0:
            print('{} percentage of nulls: {} - Total of nulls: {}'.format(index, (count / number_of_rows) * 100, count ))

In [ ]:
calculate_and_display_percentage_of_null_values(hotel_bookings_df)

Analizando la cantidad de nulos que tiene cada una de las columnas, podemos distinguir que la cantidad de nulos de las columnas 'children' y 'country' son estadisticamente despreciable. A estas filas nulas podemos eliminarlas, ya que perder un de los datos no nos va a hacer perder demasiada información. A diferencia de las columnas mencionadas anteriormente, las columnas 'agent' y 'company' tienen un porcentaje más alto de nulos que no podemos ignorar. Vamos a seguir analizando la correlación que tienen ambas columnas con la columna a predecir (en este caso 'is_canceled') para observar que tanto peso tiene en nuestro modelo.

## Análisis de correlación

In [ ]:
correlation_df = hotel_bookings_df[['is_canceled', 'agent', 'company']].corr()
sns.heatmap(correlation_df, linewidths=0.5, annot=True)

Como se ve en el Heathmap, la correlación que hay entre 'agent' y 'company' con 'is_canceled' es casi nula. Entonces, ya que estas columnas no tienen una correlación significante con la columna a predecir, decidimos eliminarlas.

## Variables con más correlación con 'is_canceled'

In [ ]:
def get_biggest_correlations_for_column(df: pd.DataFrame, column: str, max_elements: int) -> pd.DataFrame:
    corr_df = df.corr()[column].sort_values(ascending=False)
    corr_df = corr_df.iloc[1:max_elements]
    corr_df.name = f'{column}_corr'

    return corr_df

In [ ]:
correlation_series = get_biggest_correlations_for_column(df=hotel_bookings_df, column='is_canceled', max_elements=5)
correlation_series

In [ ]:
def display_scatter_plot(df: pd.DataFrame, x_column : str, y_column: str):
    df.plot.scatter(x=x_column, y=y_column)

In [ ]:
for index, _ in correlation_series.iteritems():
    display_scatter_plot(df=hotel_bookings_df, x_column=index, y_column='is_canceled')

## Limpieza de datos

Además de las columnas mencionadas anteriormente, también se elimina la columna reservation_status debido a que es un dato que se obtiene como resultado de la columna a predecir. También se hace una conversión de las columnas de categorías a representaciones numericas usando el tipo de columna Categorical.

In [ ]:
def create_category_columns(df: pd.DataFrame, list_of_columns: List[str]) -> pd.DataFrame:
    for element in list_of_columns:
        df[element] = pd.Categorical(df[element])
        df[element] = df[element].cat.codes

    return df

In [ ]:
NOT_USEFUL_COLUMNS = ['agent', 'company', 'reservation_status']
COLUMNS_WITH_NULLS_TO_DROP = ['children', 'country']
list_of_category_columns = ['is_canceled', 'meal', 'market_segment', 'reserved_room_type', 'deposit_type', 'customer_type', 'hotel', 'assigned_room_type', 'arrival_date_month', 'country', 'distribution_channel', 'reservation_status_date']


cleaned_df = hotel_bookings_df.drop(columns=NOT_USEFUL_COLUMNS)
cleaned_df = cleaned_df.dropna(subset=COLUMNS_WITH_NULLS_TO_DROP)

cleaned_df = create_category_columns(df=cleaned_df, list_of_columns=list_of_category_columns)
display(cleaned_df.isnull().sum())

## Parte 2

In [ ]:
# Separo target y features

target = 'is_canceled'
list_of_features = list(cleaned_df.columns)
list_of_features.remove(target)

In [ ]:
# Separo dataset en dataset de prueba y entrenamiento

x_train, x_test, y_train, y_test = train_test_split(cleaned_df[list_of_features].values, cleaned_df[target].values, test_size=0.2, random_state=1, stratify=cleaned_df[target].values)

In [ ]:
K_FOLDS = 10
N_ITERS = 30
kfold = StratifiedKFold(n_splits=K_FOLDS)

base_tree = DecisionTreeClassifier()


params_grid = {'criterion': ['gini', 'entropy'], 'ccp_alpha': np.linspace(0,0.05, N_ITERS), 'max_depth': list(range(1,10))}

scorer_func = make_scorer(f1_score)

### Obtención de hiperparametros

In [ ]:
randomcv = RandomizedSearchCV(estimator=base_tree, param_distributions=params_grid, scoring=scorer_func, n_iter=N_ITERS, error_score='raise', cv=kfold)
randomcv.fit(x_train, y_train)


In [ ]:
print(f'Best params: {randomcv.best_params_}')
print(f'Best score: {randomcv.best_score_}')

In [ ]:
best_tree = randomcv.best_estimator_
feat_imps = best_tree.feature_importances_

for feature_importance, feature in sorted(zip(feat_imps, list_of_features)):
    if feature_importance > 0:
        print(f'{feature}:{feature_importance}')

### Entrenamiento del modelo

In [ ]:
decision_tree = DecisionTreeClassifier().set_params(**randomcv.best_params_)
decision_tree.fit(x_train, y_train)

In [ ]:
fig = plt.figure(figsize=(50,20))
_ = plot_tree(decision_tree,
                   feature_names=list_of_features,
                   class_names=target,
                   filled=True)

### Uso el set de prueba para obtener métricas y mapa de confusión

In [ ]:
y_prediction = decision_tree.predict(x_test)

print('F1-Score: {}'.format(f1_score(y_test, y_prediction, average='binary')))

cm = confusion_matrix(y_test, y_prediction)
sns.heatmap(cm,cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

Se puede obersvar que el performance visto en el test se acerca demasiado al valor obtenido en el entrenamiento. Para realizar optimizar los hiperparámetros se utilizaron 10 folds y  al realizar la comparacíon se empleo la métrica F1-Score, ya que se calcula según el valor de la precisión y el recall, es decir, contempla el valor de dichas métricas.

### Random Forest

In [ ]:
N_ITERS = 5

rf_model = RandomForestClassifier()
randomcv_random_forest = RandomizedSearchCV(estimator=rf_model, param_distributions=params_grid, scoring=scorer_func, n_iter=N_ITERS, error_score='raise')
randomcv_random_forest.fit(x_train, y_train)

In [ ]:
print(f'Best params: {randomcv_random_forest.best_params_}')
print(f'Best score: {randomcv_random_forest.best_score_}')

In [ ]:
rf_model = rf_model.set_params(**randomcv_random_forest.best_params_)
rf_model.fit(x_train, y_train)

In [ ]:
y_prediction = rf_model.predict(x_test)

print('F1-Score: {}'.format(f1_score(y_test, y_prediction, average='binary')))

cm = confusion_matrix(y_test, y_prediction)
sns.heatmap(cm,cmap='Blues', annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

### Comparación de modelos


Random forest nos de una peor performance que un árbol de decisión usado en la parte A del ejercicio. Esta diferencia de performance puede ser causada por la cantidad de datos utilizados, ya que el modelo de Random Forest performa mejor con una mayor cantidad de datos, a diferencia del árbol de decisión que performa mejor cuando tenemos un dataset de menor tamaño.